In [3]:
import sys
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install torch


     ---------------------------------------- 0.0/172.3 MB ? eta -:--:--
     --------------------------------------- 0.8/172.3 MB 24.1 MB/s eta 0:00:08
     --------------------------------------- 1.6/172.3 MB 20.5 MB/s eta 0:00:09
      -------------------------------------- 2.7/172.3 MB 21.9 MB/s eta 0:00:08
      -------------------------------------- 4.4/172.3 MB 25.5 MB/s eta 0:00:07
     - ------------------------------------- 6.7/172.3 MB 30.6 MB/s eta 0:00:06
     -- ------------------------------------ 9.6/172.3 MB 36.2 MB/s eta 0:00:05
     --- ---------------------------------- 13.7/172.3 MB 65.6 MB/s eta 0:00:03
     --- ---------------------------------- 17.7/172.3 MB 81.8 MB/s eta 0:00:02
     ---- --------------------------------- 20.7/172.3 MB 81.8 MB/s eta 0:00:02
     ----- -------------------------------- 24.4/172.3 MB 81.8 MB/s eta 0:00:02
     ------ ------------------------------- 28.1/172.3 MB 81.8 MB/s eta 0:00:02
     ------ ------------------------------- 31.

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from sklearn.model_selection import train_test_split

In [5]:
dataset = pd.read_csv('Data/all_data.csv')

#subtract the columns L_0_x, L_0_y, L_0_z from all the columns
for i in range(0, 21):
    dataset[f'L_{i}_x'] = dataset[f'L_{i}_x'] - dataset['L_0_x']
    dataset[f'L_{i}_y'] = dataset[f'L_{i}_y'] - dataset['L_0_y']
    # dataset[f'L_{i}_z'] = dataset[f'L_{i}_z'] - dataset['L_0_z']
    dataset[f'R_{i}_x'] = dataset[f'R_{i}_x'] - dataset['R_0_x']
    dataset[f'R_{i}_y'] = dataset[f'R_{i}_y'] - dataset['R_0_y']
    # dataset[f'R_{i}_z'] = dataset[f'R_{i}_z'] - dataset['R_0_z']

#write datatset to csv
dataset.to_csv('Data/relative_data.csv', index=False)

X = dataset.iloc[:, 1:]
Y = dataset.iloc[:, 0]

# Splitting the dataset into the Training set and Test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

# # Feature Scaling
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

print(X_train)
print(Y_train)


      L_0_x  L_0_y         L_0_z     L_1_x     L_1_y     L_1_z     L_2_x  \
2739    0.0    0.0 -2.192266e-07  0.647939  0.604089 -0.003246  0.604203   
2731    0.0    0.0 -3.156693e-07  0.748733  0.501743  0.029280  0.756588   
2683    0.0    0.0 -5.226186e-07  0.744926  0.521330 -0.015679  0.697294   
446     0.0    0.0 -2.469384e-07  0.122705  0.648890 -0.032391  0.111700   
2591    0.0    0.0  3.854833e-07  0.280120  0.577589  0.002965  0.282810   
...     ...    ...           ...       ...       ...       ...       ...   
1940    0.0    0.0  2.630914e-07  0.332738  0.727581 -0.019074  0.363878   
1796    0.0    0.0  5.141460e-07  0.324174  0.701451 -0.023089  0.365412   
2271    0.0    0.0  3.167221e-07  0.365823  0.713335 -0.021838  0.408096   
165     0.0    0.0 -2.282169e-07  0.180145  0.444481 -0.022140  0.187349   
566     0.0    0.0 -7.696020e-08  0.190667  0.364593 -0.021365  0.212868   

         L_2_y     L_2_z     L_3_x  ...    R_17_z    R_18_x    R_18_y  \
2739  0.527084